# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import numpy as np

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.


In [7]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.set_index('id_cliente', inplace=True)
X = pd.get_dummies(df.drop(['renda', 'data_ref'], axis=1), drop_first=True)
y = df['renda']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [9]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_results = {}

for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    ridge_results[alpha] = r2_score(y_test, y_pred)

best_ridge_alpha = max(ridge_results, key=ridge_results.get)
print(f'Melhor alpha para Ridge: {best_ridge_alpha}, R^2: {ridge_results[best_ridge_alpha]}')


Melhor alpha para Ridge: 0.1, R^2: 0.26903183169156664


In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lasso_cv = LassoCV(alphas=alphas[1:], cv=5, random_state=42, max_iter=10000) 
lasso_cv.fit(X_train_scaled, y_train)


y_pred_lasso_cv = lasso_cv.predict(X_test_scaled)
r2_score_lasso_cv = r2_score(y_test, y_pred_lasso_cv)

print(f'Melhor alpha para LASSO com CV: {lasso_cv.alpha_}, R^2: {r2_score_lasso_cv}')



Melhor alpha para LASSO com CV: 0.1, R^2: 0.2690296089628653


In [13]:
linear_regression = LinearRegression()


sfs = SFS(linear_regression,
          k_features='best',  
          forward=True,  
          floating=False,
          scoring='r2',
          cv=0)  


sfs.fit(X_train_scaled, y_train)

best_features = list(sfs.k_feature_names_)  
X_train_sfs = sfs.transform(X_train_scaled)
X_test_sfs = sfs.transform(X_test_scaled)
linear_regression.fit(X_train_sfs, y_train)
y_pred_sfs = linear_regression.predict(X_test_sfs)
r2_score_sfs = r2_score(y_test, y_pred_sfs)

print(f'Melhor número de features: {len(best_features)}, R^2: {r2_score_sfs}')

Melhor número de features: 24, R^2: 0.2690308242376487


In [ ]:
5. Nesse caso melhor modelo seria o que posso o melhor R^2  que nesse casos é Ridge: 0.1, R^2: 0.26903183169156664

In [23]:

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)


lasso = Lasso(alpha=0.01, max_iter=10000)


lasso.fit(X_train_poly, y_train)
y_pred_poly = lasso.predict(X_test_poly)


r2 = r2_score(y_test, y_pred_poly)
print("R^2 com transformação polinomial:", r2)



R^2 com transformação polinomial: 0.3758945455614586


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.256e+11, tolerance: 7.723e+07
  model = cd_fast.enet_coordinate_descent(


In [17]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=5, random_state=42)  
tree.fit(X_train_scaled, y_train)
y_pred_tree = tree.predict(X_test_scaled)
r2_score_tree = r2_score(y_test, y_pred_tree)

print("R^2 com árvore de regressão:", r2_score_tree)


R^2 com árvore de regressão: 0.38372219439238775
